In [ ]:
import numpy as np
import pandas as pd
from pandas import DataFrame as DF
from sklearn.ensemble import RandomForestRegressor as RF
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
from lightgbm import LGBMRegressor 

In [ ]:
xtrain=pd.read_csv((r'C:\Users\86153\Desktop\house-prices-advanced-regression-techniques\xtrain.csv'),index_col=[0])
xtest=pd.read_csv((r'C:\Users\86153\Desktop\house-prices-advanced-regression-techniques\xtest.csv'),index_col=[0])
ytrain=pd.read_csv((r'C:\Users\86153\Desktop\house-prices-advanced-regression-techniques\ytrain.csv'),index_col=[0])
ytest=pd.read_csv((r'C:\Users\86153\Desktop\house-prices-advanced-regression-techniques\ytest.csv'),index_col=[0])
y=pd.read_csv((r'C:\Users\86153\Desktop\house-prices-advanced-regression-techniques\y.csv'),index_col=[0])
feature=pd.read_csv((r'C:\Users\86153\Desktop\house-prices-advanced-regression-techniques\feature.csv'),index_col=[0])
target=pd.read_csv((r'C:\Users\86153\Desktop\house-prices-advanced-regression-techniques\target.csv'),index_col=[0])

In [ ]:
ytest=ytest['SalePrice']
ytrain=ytrain['SalePrice']
target=target['SalePrice']

In [ ]:
def rmse(predict_,true_,len_):
    return (((np.log(predict_)- np.log(true_))**2).sum()/len_)**0.5

#开始建模

In [ ]:
#随机森林
rf=RF(n_estimators=73,random_state=66,max_depth=19,criterion='mae',min_samples_leaf=1,max_features=80)
rf.fit(xtrain,ytrain)
pre=rf.predict(xtest)
print(rmse(pre,ytest,len(pre)))
#0.1368336187504412

In [ ]:
#ridge回归
rg=Ridge(alpha=15.52631579,random_state=66,fit_intercept=False)
rg.fit(xtrain,ytrain)
preri=rg.predict(xtest)
rmse(abs(preri),ytest,len(preri))
#0.15864391482319576

In [ ]:
#lasso回归
lasso=Lasso(alpha=97,random_state=66,max_iter=95,positive=True)
lasso.fit(xtrain,ytrain)
pre1=lasso.predict(xtest)
rmse(abs(pre1),ytest,len(ytest))
#0.15432119063694827

In [ ]:
#linear核svr
svr=SVR(kernel='linear',epsilon=20,C=655)#0.14573277859081937
svr.fit(xtrain,ytrain)
pre1=svr.predict(xtest)
rmse(pre1,ytest,len(pre1))

In [ ]:
#poly核svr
svr=SVR(kernel='poly',degree=1,gamma=1.9375,coef0=89.79591837,C=311)
svr.fit(xtrain,ytrain)
pre1=svr.predict(xtest)
rmse(pre1,ytest,len(pre1))
#0.1454503415294951

In [ ]:
#lgbm
params={}
lgbm=LGBMRegressor(boosting_type='gbdt',application='regression',n_estimators=76
                  ,learning_rate=0.06142,max_depth=21,num_leaves=16,subsample=0.00011
                  ,colsample_bytree=1,reg_alpha=70,reg_lambda=0,num_iterations=91,)
gsearch = GridSearchCV(lgbm, param_grid=params, scoring='neg_mean_squared_error', cv=5,n_jobs=-1)
gsearch.fit(xtrain,ytrain)
print("Best score: %0.3f" % gsearch.best_score_)
print(gsearch.best_params_)
pre=gsearch.predict(xtest)
rmse(pre,ytest,len(pre))


In [ ]:
#以下是最后部分

In [ ]:
#先用训练集和测试集算出最佳的每个模型系数再组装一个简单的集成模型
rf=RF(n_estimators=73,random_state=66,max_depth=19,criterion='mae',min_samples_leaf=1,max_features=80)
rf.fit(xtrain,ytrain)
rf_pre1=rf.predict(xtest)
print('rf')

rg=Ridge(alpha=15.52631579,random_state=66,fit_intercept=False)
rg.fit(xtrain,ytrain)
ridge_pre1=rg.predict(xtest)
print('ridge')

lasso=Lasso(alpha=97,random_state=66,max_iter=95,positive=True)
lasso.fit(xtrain,ytrain)
lasso_pre1=lasso.predict(xtest)
print('lasso')

svr=SVR(kernel='linear',epsilon=20,C=655)#0.14573277859081937#[0.1522173355008953]
svr.fit(xtrain,ytrain)
linearsvr_pre1=svr.predict(xtest)
print('linearsvr')

polysvr=SVR(kernel='poly',degree=1,gamma=1.9375,coef0=89.79591837,C=311)#0.14673301057055446
polysvr.fit(xtrain,ytrain)
polysvr_pre1=polysvr.predict(xtest)
print('polysvr')

lgbm=LGBMRegressor(nthread=-1,boosting_type='gbdt',application='regression',max_depth=8,bagging_freq=1,reg_alpha=7,reg_lambda=2,n_estimators=46,learning_rate=0.11421052631578943)
lgbm.fit(xtrain,ytrain)
lgbm_pre1=lgbm.predict(xtest)
print('lgbm')

In [ ]:
score=[]
for i1 in range(0,10,1):
    a=i1*0.1
    a=round(a,2)
    for i2 in range(0,10-i1):
        b=i2*0.1
        b=round(b,2)
        for i3 in range(0,10-i1-i2):
            c=i3*0.1
            c=round(c,2)
            for i4 in range(0,10-i1-i2-i3):
                d=i4*0.1
                d=round(d,2)
                for i5 in range(0,10-i1-i2-i3-i4):
                    e=i5/10
                    e=round(e,2)
                    f=(10-i1-i2-i3-i4-i5)*0.1
                    f=round(f,2)
                    result=rf_pre1*a+lgbm_pre1*b+ridge_pre1*c+polysvr_pre1*d+linearsvr_pre1*e+lasso_pre1*f
                    score.append(rmse(result,ytest,len(result)))
                    print(rmse(result,ytest,len(result)),a,b,c,d,e,f)
                        

min(score)

In [ ]:
#得到系数后先训练基模型，再用上面得到的每个模型系数进行组合
rf=RF(n_estimators=73,random_state=66,max_depth=19,criterion='mae',min_samples_leaf=1,max_features=80)
rf.fit(feature,target)
rf_pre=rf.predict(y)
print('rf')

rg=Ridge(alpha=15.52631579,random_state=66,fit_intercept=False)
rg.fit(feature,target)
ridge_pre=rg.predict(y)
print('ridge')

lasso=Lasso(alpha=97,random_state=66,max_iter=95,positive=True)
lasso.fit(feature,target)
lasso_pre=lasso.predict(y)
print('lasso')

svr=SVR(kernel='linear',epsilon=20,C=655)#0.14573277859081937#[0.1522173355008953]
svr.fit(feature,target)
linearsvr_pre=svr.predict(y)
print('linearsvr')

svr=SVR(kernel='poly',degree=1,gamma=1.9375,coef0=89.79591837,C=311)#0.14673301057055446
svr.fit(feature,target)
polysvr_pre=svr.predict(y)
print('polysvr')
lgbm=LGBMRegressor(nthread=-1,boosting_type='gbdt',application='regression'
,max_depth=8,bagging_freq=1,reg_alpha=7,reg_lambda=2,n_estimators=46,learning_rate=0.11421052631578943)
lgbm.fit(feature,target)
lgbm_pre=lgbm.predict(y)
print('lgbm')

In [ ]:
result=rf_pre*0.4+lgbm_pre*0.3+polysvr_pre*0.1+lasso_pre*0.2

In [ ]:
a=DF({'id':range(1461,2920),'SalePrice':result},columns=['id','SalePrice'])
a.set_index(['id'],inplace=True)
a.to_csv(r'C:\Users\86153\Desktop\house-prices-advanced-regression-techniques\jieguo_mix.csv')